## 1️⃣ Initialisation - Clone du Repo

## 1️⃣ Initialisation - Clone du Repo

In [8]:
import os
import subprocess

# Clone repo PER (adapte pour Kaggle/Colab/local)
repo_path = "/kaggle/working/PER" if "/kaggle" in os.getcwd() else "C:\\Users\\mathy\\Documents\\SI5\\PER"

if os.path.exists(repo_path):
    print(f"✓ Repo trouvé: {repo_path}")
    os.chdir(repo_path)
else:
    print(f"Clonage du repo...")
    subprocess.run(["git", "clone", "https://github.com/MathysGallay/PER.git", repo_path], check=True)
    os.chdir(repo_path)

print(f"📁 Répertoire actuel: {os.getcwd()}")

✓ Repo trouvé: C:\Users\mathy\Documents\SI5\PER
📁 Répertoire actuel: C:\Users\mathy\Documents\SI5\PER


## 2️⃣ Préparation du Dataset

In [9]:
# Exécuter les scripts de préparation
print("🔄 Exécution des scripts de préparation...\n")

!python download_mini_coco_animals.py
!python image_thermal_converter.py
!python thermal_augmentation.py

print("\n✓ Préparation terminée")

🔄 Exécution des scripts de préparation...

Extraction annotations...
Nombre d'images trouvées : 25465
Images téléchargées : 0
Mini-COCO (animaux uniquement) généré



Téléchargement images: 100%|██████████| 118287/118287 [00:00<00:00, 1302322.92it/s]

Conversion RGB → Thermal:   0%|          | 0/500 [00:00<?, ?it/s]
Traceback (most recent call last):
  File "C:\Users\mathy\Documents\SI5\PER\image_thermal_converter.py", line 69, in <module>
    gray, thermal_color = rgb_to_thermal(img_path)
  File "C:\Users\mathy\Documents\SI5\PER\image_thermal_converter.py", line 50, in rgb_to_thermal
    gray = np.clip(alpha * gray + beta, 0, 255).astype(np.uint8)
  File "c:\Users\mathy\anaconda3\envs\PER_GPU\lib\site-packages\numpy\core\fromnumeric.py", line 2169, in clip
    return _wrapfunc(a, 'clip', a_min, a_max, out=out, **kwargs)
  File "c:\Users\mathy\anaconda3\envs\PER_GPU\lib\site-packages\numpy\core\fromnumeric.py", line 59, in _wrapfunc
    return bound(*args, **kwds)
  File "c:\Users\mathy\anaconda3\envs\PER_GPU\lib\site-packages\numpy\_core\_methods.py", line 15, in <module>
    from numpy._core import numerictypes as nt
  File "c:\Users\mathy\anacon


✓ Préparation terminée



Augmentation thermique:   0%|          | 0/500 [00:00<?, ?it/s]
Traceback (most recent call last):
  File "c:\Users\mathy\anaconda3\envs\PER_GPU\lib\site-packages\numpy\_core\arrayprint.py", line 34, in <module>
    from . import numerictypes as _nt
  File "c:\Users\mathy\anaconda3\envs\PER_GPU\lib\site-packages\numpy\_core\numerictypes.py", line 102, in <module>
    from ._type_aliases import (
  File "c:\Users\mathy\anaconda3\envs\PER_GPU\lib\site-packages\numpy\_core\_type_aliases.py", line 38, in <module>
    allTypes[_abstract_type_name] = getattr(ma, _abstract_type_name)
AttributeError: module 'numpy.core.multiarray' has no attribute 'generic'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\Users\mathy\Documents\SI5\PER\thermal_augmentation.py", line 52, in <module>
    aug = augment(img_color)
  File "C:\Users\mathy\Documents\SI5\PER\thermal_augmentation.py", line 37, in augment
    img = np.clip(img.astype(np

In [10]:
import torch

print("🖥️ Vérification GPU:")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

🖥️ Vérification GPU:
CUDA available: True
GPU: NVIDIA GeForce RTX 4060 Laptop GPU


In [11]:
import os
import glob
import random
import shutil

# Chemins du dataset
SOURCE_DIR = "thermal_coco/augmented"
SOURCE_LABELS = "mini_coco/labels"
TARGET_DIR = "thermal_yolo"

print(f"Source exists: {os.path.exists(SOURCE_DIR)}")
print(f"Labels exist: {os.path.exists(SOURCE_LABELS)}")

os.makedirs(TARGET_DIR, exist_ok=True)

# Créer structure YOLO
folders = ["images/train", "images/val", "labels/train", "labels/val"]
for f in folders:
    os.makedirs(os.path.join(TARGET_DIR, f), exist_ok=True)

# Récupérer images et créer splits
images = glob.glob(os.path.join(SOURCE_DIR, "*.jpg"))
print(f"Images trouvées: {len(images)}") 

if len(images) > 0:
    train_ratio = 0.8
    random.shuffle(images)
    split = int(len(images) * train_ratio)
    
    train_imgs = images[:split]
    val_imgs = images[split:]
    
    print(f"Train: {len(train_imgs)}, Val: {len(val_imgs)}") 
    
    # Copier images et labels vers structure YOLO
    for split_name, img_list, target_split in [("train", train_imgs, "train"), ("val", val_imgs, "val")]:
        for img_path in img_list:
            img_name = os.path.basename(img_path)
            shutil.copy(img_path, os.path.join(TARGET_DIR, f"images/{target_split}", img_name))
            
            base_name = img_name.split('_aug')[0] if '_aug' in img_name else img_name.replace('.jpg', '')
            label_src = os.path.join(SOURCE_LABELS, base_name + '.txt')
            label_dst = os.path.join(TARGET_DIR, f"labels/{target_split}", img_name.replace('.jpg', '.txt'))
            
            if os.path.exists(label_src):
                shutil.copy(label_src, label_dst)
    
    print(f"✓ Dataset YOLO créé")
else:
    print("❌ Aucune image trouvée!")

Source exists: True
Labels exist: True
Images trouvées: 2500
Train: 2000, Val: 500
✓ Dataset YOLO créé


In [12]:
# Définir classes et générer data.yaml
ANIMAL_CLASSES = {
    14: "bird",
    15: "cat",
    16: "dog",
    17: "horse",
    18: "sheep",
    19: "cow",
    20: "elephant",
    21: "bear",
    22: "zebra",
    23: "giraffe",
}

classes = list(ANIMAL_CLASSES.values())
print(f"Classes: {classes}")

yaml_content = f"""train: {TARGET_DIR}/images/train
val: {TARGET_DIR}/images/val

nc: {len(classes)}
names: {classes}
"""

with open("data.yaml", "w") as f:
    f.write(yaml_content)

print("✓ data.yaml généré")

Classes: ['bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe']
✓ data.yaml généré


## 3️⃣ Installation Dépendances

In [13]:
print("--- 1. DÉSINSTALLATION DES PAQUETS EN CONFLIT ---") 
!pip uninstall numpy matplotlib ultralytics -y 2>/dev/null

print("\n--- 2. INSTALLATION DES VERSIONS COMPATIBLES ---")
!pip install numpy==1.26.4
!pip install ultralytics
!pip install matplotlib

--- 1. DÉSINSTALLATION DES PAQUETS EN CONFLIT ---

--- 2. INSTALLATION DES VERSIONS COMPATIBLES ---


Le chemin d'acc�s sp�cifi� est introuvable.


## 4️⃣ Entraînement Optimisé avec Ultralytics

In [14]:
from ultralytics import YOLO
import os

# === PARAMÈTRES OPTIMISÉS ===
MODEL_NAME = 'yolov8s.pt'      # Modèle s (7.2M params) au lieu de n
RUN_NAME = 'thermal_exp_improved'
IMG_SIZE = 416                 # Résolution augmentée
BATCH_SIZE = 8                 # Réduit pour résolution 416
EPOCHS = 100                   # Convergence complète
PATIENCE = 10                  # Early stopping
DEVICE = 0                     # GPU

# Charger modèle
model = YOLO(MODEL_NAME)

print(f"🚀 Entraînement optimisé: {MODEL_NAME}")
print(f"   Résolution: {IMG_SIZE}x{IMG_SIZE}")
print(f"   Batch: {BATCH_SIZE}, Epochs: {EPOCHS}, Patience: {PATIENCE}\n")

# Entraîner
results = model.train(
    data='data.yaml',
    imgsz=IMG_SIZE,
    batch=BATCH_SIZE,
    epochs=EPOCHS,
    patience=PATIENCE,
    save_period=10,
    name=RUN_NAME,
    project='runs/train',
    device=DEVICE,
    cache=True,
    exist_ok=True,
    amp=False
)

BEST_WEIGHTS_PATH = os.path.join(results.save_dir, 'weights', 'best.pt')
print(f"\n✅ Entraînement terminé")
print(f"Modèle: {BEST_WEIGHTS_PATH}")

🚀 Entraînement optimisé: yolov8s.pt
   Résolution: 416x416
   Batch: 8, Epochs: 100, Patience: 10

New https://pypi.org/project/ultralytics/8.3.240 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.233  Python-3.10.19 torch-2.5.1 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
engine\trainer: agnostic_nms=False, amp=False, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=416, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s

error: OpenCV(4.12.0) :-1: error: (-5:Bad argument) in function 'imdecode'
> Overload resolution failed:
>  - buf is not a numpy array, neither a scalar
>  - Expected Ptr<cv::UMat> for argument 'buf'


## 5️⃣ Export ONNX

In [ ]:
# Charger meilleur modèle et exporter
best_model = YOLO(BEST_WEIGHTS_PATH)

print("📦 Export ONNX...")
path_to_onnx = best_model.export(format='onnx', imgsz=IMG_SIZE)

print(f"✅ Export terminé")
print(f"ONNX: {path_to_onnx}")

## 📹 Alternative - Extraction YouTube (Optionnel)

In [ ]:
# Installation dépendances YouTube (optionnel)
# !pip install yt-dlp opencv-python-headless

import os
import cv2
import subprocess

def extract_frames_from_youtube(video_url, output_dir="real_thermal_dataset", fps=0.5, max_frames=300):
    """Extrait frames d'une vidéo YouTube"""
    os.makedirs(output_dir, exist_ok=True)
    
    video_path = os.path.join(output_dir, "temp_video.mp4")
    print(f"📥 Téléchargement...")
    try:
        subprocess.run([
            "yt-dlp", 
            "-f", "best[height<=720]", 
            "-o", video_path, 
            video_url
        ], check=True)
    except FileNotFoundError:
        print("❌ yt-dlp non installé: pip install yt-dlp")
        return 0
    
    print(f"🎞️ Extraction (FPS={fps})...")
    cap = cv2.VideoCapture(video_path)
    video_fps = cap.get(cv2.CAP_PROP_FPS)
    frame_interval = max(1, int(video_fps / fps))
    
    frame_count = 0
    saved_count = 0
    
    while cap.isOpened() and saved_count < max_frames:
        ret, frame = cap.read()
        if not ret:
            break
        
        if frame_count % frame_interval == 0:
            frame_filename = os.path.join(output_dir, f"frame_{saved_count:05d}.jpg")
            cv2.imwrite(frame_filename, frame, [cv2.IMWRITE_JPEG_QUALITY, 95])
            saved_count += 1
            if saved_count % 50 == 0:
                print(f"  ✓ {saved_count} frames...")
        frame_count += 1
    
    cap.release()
    os.remove(video_path)
    print(f"✅ {saved_count} frames dans {output_dir}")
    return saved_count

# Utilisation (décommenter et ajouter URLs)
# YOUTUBE_URLS = ["https://www.youtube.com/watch?v=..."]
# for i, url in enumerate(YOUTUBE_URLS):
#     extract_frames_from_youtube(url, f"real_thermal_dataset/video_{i+1}")

In [ ]:
import os
import subprocess

# Clone repo PER (adapte pour Kaggle/Colab/local)
repo_path = "/kaggle/working/PER" if "/kaggle" in os.getcwd() else "C:\\Users\\mathy\\Documents\\SI5\\PER"

if os.path.exists(repo_path):
    print(f"✓ Repo trouvé: {repo_path}")
    os.chdir(repo_path)
else:
    print(f"Clonage du repo...")
    subprocess.run(["git", "clone", "https://github.com/MathysGallay/PER.git", repo_path], check=True)
    os.chdir(repo_path)

print(f"📁 Répertoire actuel: {os.getcwd()}")

✓ Repo trouvé: C:\Users\mathy\Documents\SI5\PER
📁 Répertoire actuel: C:\Users\mathy\Documents\SI5\PER


## 2️⃣ Préparation du Dataset

In [ ]:
# Exécuter les scripts de préparation
print("🔄 Exécution des scripts de préparation...\n")

!python download_mini_coco_animals.py
!python image_thermal_converter.py
!python thermal_augmentation.py

print("\n✓ Préparation terminée")

🔄 Exécution des scripts de préparation...

Extraction annotations...
Nombre d'images trouvées : 25465
Images téléchargées : 0
Mini-COCO (animaux uniquement) généré



Téléchargement images: 100%|██████████| 118287/118287 [00:00<00:00, 825315.21it/s]


Conversion thermique terminée !
Images enregistrées dans thermal_coco/images



Conversion RGB → Thermal: 100%|██████████| 500/500 [00:11<00:00, 42.66it/s]


Augmentation thermique terminée !

✓ Préparation terminée



Augmentation thermique: 100%|██████████| 500/500 [00:24<00:00, 20.41it/s]


In [ ]:
import torch

print("🖥️ Vérification GPU:")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

🖥️ Vérification GPU:
CUDA available: True
GPU: NVIDIA GeForce RTX 4060 Laptop GPU


In [ ]:
import os
import glob
import random
import shutil

# Chemins du dataset
SOURCE_DIR = "thermal_coco/augmented"
SOURCE_LABELS = "mini_coco/labels"
TARGET_DIR = "thermal_yolo"

print(f"Source exists: {os.path.exists(SOURCE_DIR)}")
print(f"Labels exist: {os.path.exists(SOURCE_LABELS)}")

os.makedirs(TARGET_DIR, exist_ok=True)

# Créer structure YOLO
folders = ["images/train", "images/val", "labels/train", "labels/val"]
for f in folders:
    os.makedirs(os.path.join(TARGET_DIR, f), exist_ok=True)

# Récupérer images et créer splits
images = glob.glob(os.path.join(SOURCE_DIR, "*.jpg"))
print(f"Images trouvées: {len(images)}") 

if len(images) > 0:
    train_ratio = 0.8
    random.shuffle(images)
    split = int(len(images) * train_ratio)
    
    train_imgs = images[:split]
    val_imgs = images[split:]
    
    print(f"Train: {len(train_imgs)}, Val: {len(val_imgs)}") 
    
    # Copier images et labels vers structure YOLO
    for split_name, img_list, target_split in [("train", train_imgs, "train"), ("val", val_imgs, "val")]:
        for img_path in img_list:
            img_name = os.path.basename(img_path)
            shutil.copy(img_path, os.path.join(TARGET_DIR, f"images/{target_split}", img_name))
            
            base_name = img_name.split('_aug')[0] if '_aug' in img_name else img_name.replace('.jpg', '')
            label_src = os.path.join(SOURCE_LABELS, base_name + '.txt')
            label_dst = os.path.join(TARGET_DIR, f"labels/{target_split}", img_name.replace('.jpg', '.txt'))
            
            if os.path.exists(label_src):
                shutil.copy(label_src, label_dst)
    
    print(f"✓ Dataset YOLO créé")
else:
    print("❌ Aucune image trouvée!")

Source exists: True
Labels exist: True
Images trouvées: 2500
Train: 2000, Val: 500
✓ Dataset YOLO créé


In [ ]:
# Définir classes et générer data.yaml
ANIMAL_CLASSES = {
    14: "bird",
    15: "cat",
    16: "dog",
    17: "horse",
    18: "sheep",
    19: "cow",
    20: "elephant",
    21: "bear",
    22: "zebra",
    23: "giraffe",
}

classes = list(ANIMAL_CLASSES.values())
print(f"Classes: {classes}")

yaml_content = f"""train: {TARGET_DIR}/images/train
val: {TARGET_DIR}/images/val

nc: {len(classes)}
names: {classes}
"""

with open("data.yaml", "w") as f:
    f.write(yaml_content)

print("✓ data.yaml généré")

Classes: ['bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe']
✓ data.yaml généré


## 3️⃣ Installation Dépendances

In [ ]:
print("--- 1. DÉSINSTALLATION DES PAQUETS EN CONFLIT ---") 
!pip uninstall numpy matplotlib ultralytics -y 2>/dev/null

print("\n--- 2. INSTALLATION DES VERSIONS COMPATIBLES ---")
!pip install numpy==1.26.4
!pip install ultralytics
!pip install matplotlib

--- 1. DÉSINSTALLATION DES PAQUETS EN CONFLIT ---

--- 2. INSTALLATION DES VERSIONS COMPATIBLES ---


Le chemin d'acc�s sp�cifi� est introuvable.


  Using cached numpy-1.26.4-cp310-cp310-win_amd64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp310-cp310-win_amd64.whl (15.8 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.3
    Uninstalling numpy-1.24.3:
      Successfully uninstalled numpy-1.24.3


  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
onnx-ir 0.1.12 requires onnx>=1.16, but you have onnx 1.14.1 which is incompatible.
onnx-ir 0.1.12 requires typing_extensions>=4.10, but you have typing-extensions 4.8.0 which is incompatible.
onnxscript 0.5.6 requires onnx>=1.16, but you have onnx 1.14.1 which is incompatible.
onnxscript 0.5.6 requires typing_extensions>=4.10, but you have typing-extensions 4.8.0 which is incompatible.
tensorflow-cpu 2.19.1 requires keras>=3.5.0, but you have keras 2.13.1 which is incompatible.
tensorflow-cpu 2.19.1 requires tensorboard~=2.19.0, but you have tensorboard 2.13.0 which is incompatible.
tensorflow-intel 2.13.1 requires numpy<=1.24.3,>=1.22, but you have numpy 1.26.4 which is incompatible.
tensorflow-intel 2.13.1 requires typing-extensions<4.6.0,>=3.6.6, but you have typing-extensi

## 4️⃣ Entraînement Optimisé avec Ultralytics

In [ ]:
from ultralytics import YOLO
import os

# === PARAMÈTRES OPTIMISÉS ===
MODEL_NAME = 'yolov8s.pt'      # Modèle s (7.2M params) au lieu de n
RUN_NAME = 'thermal_exp_improved'
IMG_SIZE = 416                 # Résolution augmentée
BATCH_SIZE = 8                 # Réduit pour résolution 416
EPOCHS = 100                   # Convergence complète
PATIENCE = 10                  # Early stopping
DEVICE = 0                     # GPU

# Charger modèle
model = YOLO(MODEL_NAME)

print(f"🚀 Entraînement optimisé: {MODEL_NAME}")
print(f"   Résolution: {IMG_SIZE}x{IMG_SIZE}")
print(f"   Batch: {BATCH_SIZE}, Epochs: {EPOCHS}, Patience: {PATIENCE}\n")

# Entraîner
results = model.train(
    data='data.yaml',
    imgsz=IMG_SIZE,
    batch=BATCH_SIZE,
    epochs=EPOCHS,
    patience=PATIENCE,
    save_period=10,
    name=RUN_NAME,
    project='runs/train',
    device=DEVICE,
    cache=True,
    exist_ok=True,
    amp=False
)

BEST_WEIGHTS_PATH = os.path.join(results.save_dir, 'weights', 'best.pt')
print(f"\n✅ Entraînement terminé")
print(f"Modèle: {BEST_WEIGHTS_PATH}")

🚀 Entraînement optimisé: yolov8s.pt
   Résolution: 416x416
   Batch: 8, Epochs: 100, Patience: 10

New https://pypi.org/project/ultralytics/8.3.240 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.233  Python-3.10.19 torch-2.5.1 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)


ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

## 5️⃣ Export ONNX

In [ ]:
# Charger meilleur modèle et exporter
best_model = YOLO(BEST_WEIGHTS_PATH)

print("📦 Export ONNX...")
path_to_onnx = best_model.export(format='onnx', imgsz=IMG_SIZE)

print(f"✅ Export terminé")
print(f"ONNX: {path_to_onnx}")

## 📹 Alternative - Extraction YouTube (Optionnel)

In [ ]:
# Installation dépendances YouTube (optionnel)
# !pip install yt-dlp opencv-python-headless

import os
import cv2
import subprocess

def extract_frames_from_youtube(video_url, output_dir="real_thermal_dataset", fps=0.5, max_frames=300):
    """Extrait frames d'une vidéo YouTube"""
    os.makedirs(output_dir, exist_ok=True)
    
    video_path = os.path.join(output_dir, "temp_video.mp4")
    print(f"📥 Téléchargement...")
    try:
        subprocess.run([
            "yt-dlp", "-f", "best[height<=720]", "-o", video_path, video_url
        ], check=True)
    except FileNotFoundError:
        print("❌ yt-dlp non installé: pip install yt-dlp")
        return 0
    
    print(f"🎞️ Extraction (FPS={fps})...")
    cap = cv2.VideoCapture(video_path)
    video_fps = cap.get(cv2.CAP_PROP_FPS)
    frame_interval = max(1, int(video_fps / fps))
    
    frame_count = 0
    saved_count = 0
    
    while cap.isOpened() and saved_count < max_frames:
        ret, frame = cap.read()
        if not ret:
            break
        
        if frame_count % frame_interval == 0:
            frame_filename = os.path.join(output_dir, f"frame_{saved_count:05d}.jpg")
            cv2.imwrite(frame_filename, frame, [cv2.IMWRITE_JPEG_QUALITY, 95])
            saved_count += 1
            if saved_count % 50 == 0:
                print(f"  ✓ {saved_count} frames...")
        frame_count += 1
    
    cap.release()
    os.remove(video_path)
    print(f"✅ {saved_count} frames dans {output_dir}")
    return saved_count

# Utilisation (décommenter et ajouter URLs)
# YOUTUBE_URLS = ["https://www.youtube.com/watch?v=..."]
# for i, url in enumerate(YOUTUBE_URLS):
#     extract_frames_from_youtube(url, f"real_thermal_dataset/video_{i+1}")